In [ ]:
def create_data_structure(color_spaces, ppm_values, specimens_per_ppm, metrics_info):
    data_structure = {}
    for color_space in color_spaces:
        data_structure[color_space] = {}
        for ppm in ppm_values:
            data_structure[color_space][ppm] = {}
            for specimen in range(1, specimens_per_ppm + 1):
                data_structure[color_space][ppm][specimen] = {}
                for metric, submetrics in metrics_info.items():
                    if isinstance(submetrics, dict):
                        data_structure[color_space][ppm][specimen][metric] = {k: None for k in submetrics}
                    else:
                        data_structure[color_space][ppm][specimen][metric] = None
    return data_structure


color_spaces = ['Hsv', 'Yiq', 'Lab']
ppm_values = [9, 16, 25, 36] 
specimens_per_ppm = 5 


metrics_info = {
    'Key Metric 1': {'C1': None, 'C2': None, 'C3': None},
    'Key Metric 2': 'Delta E 2000 Success Rate',
    'Key Metric 3': {
        'C1': {'Mean': None, 'Std': None, 'True Mean': None, 'True Std': None},
        'C2': {'Mean': None, 'Std': None, 'True Mean': None, 'True Std': None},
        'C3': {'Mean': None, 'Std': None, 'True Mean': None, 'True Std': None}
    },
    'Key Metric 4': {
        'Row Mean': None, 'Column Mean': None, 'True Row Mean': None, 'True Column Mean': None
    },
    'Key Metric 5': 'Perfect Match Rate', 
    'Key Metric 6': 'Threshold Success Rate' 
}

data_structure = create_data_structure(color_spaces, ppm_values, specimens_per_ppm, metrics_info)


In [ ]:

from matplotlib import pyplot as plt
import numpy as np
from colorsys import hsv_to_rgb 
from PIL import Image
from transformers import pipeline
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

from colormath.color_objects import LabColor, AdobeRGBColor
from colormath.color_conversions import convert_color
from colorsys import hsv_to_rgb, yiq_to_rgb, rgb_to_hsv
from colorsys import rgb_to_yiq
from CQRCode import CQrCode

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import color
from skimage.color import deltaE_ciede2000
from skimage.util import img_as_ubyte

def calculate_rrmse(predicted, true):
    errors = predicted - true
    if true != 0:
        squared_relative_errors = (errors / true) ** 2
        mean_squared_relative_errors = np.mean(squared_relative_errors)
        rrmse = np.sqrt(mean_squared_relative_errors)
        return rrmse
    else:
        pass

def calculate_rmse(predicted, true):
    errors = predicted - true
    squared_errors = errors ** 2
    mean_squared_errors = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_errors)
    
    return rmse
def delta_e_distance(c1, c2):
    return deltaE_ciede2000(c1, c2)

def quantize_using_delta_e(img_array, palette):
    lab_image = color.rgb2lab(img_array)
    lab_palette = color.rgb2lab(palette.reshape(-1, 1, 3))
    pixel_array = lab_image.reshape((-1, 1, 3))
    quantized_indices = np.zeros(len(pixel_array), dtype=int)
    for i in range(len(pixel_array)):
        distances = [delta_e_distance(pixel_array[i], lab_color) for lab_color in lab_palette]
        quantized_indices[i] = np.argmin(distances)

    quantized_array = palette[quantized_indices]
    return quantized_array.reshape(img_array.shape)

def rmse(true, pred):
    return np.sqrt(((pred - true) ** 2).mean())

def rrmse(true, pred):
    num = np.sum(np.square(true - pred)) / true.size
    den = np.sum(np.square(pred))
    squared_error = num/den
    rrmse_loss = np.sqrt(squared_error)
    return rrmse_loss*100

def to_hsv(rgb_triplet):
    r, g, b = [x / 255 for x in rgb_triplet] 
    h, s, v = rgb_to_hsv(r, g, b)
    return (h, s, v)

def to_lab(rgb_triplet):
    r, g, b = [x / 255 for x in rgb_triplet] 
    rgb_color = AdobeRGBColor(r, g, b)
    lab_color = convert_color(rgb_color, LabColor)
    return (lab_color.lab_l, lab_color.lab_a, lab_color.lab_b)

def to_yiq(rgb_triplet):
    r, g, b = [x / 255 for x in rgb_triplet] 
    y, i, q = rgb_to_yiq(r, g, b)
    return (y, i, q)

def rrmse_errors(color_space, array1, array2):
    if color_space == "Yiq":
        true = np.array([to_yiq(color) for color in array2])
        pred = np.array([to_yiq(color) for color in array1])  
    elif color_space == "Hsv":
        true = np.array([to_hsv(color) for color in array2])
        pred = np.array([to_hsv(color) for color in array1])  
    elif color_space == "Lab":
        true = np.array([to_lab(color) for color in array2])
        pred = np.array([to_lab(color) for color in array1]) 
    else:
        pass          
    c1_rrmse = [rrmse(true[i][0], pred[i][0]) for i in range(len(true))]
    c2_rrmse = [rrmse(true[i][1], pred[i][1]) for i in range(len(true))]
    c3_rrmse = [rrmse(true[i][2], pred[i][2]) for i in range(len(true))]
    c1_rrmse_avg = np.mean(np.ma.masked_invalid(c1_rrmse))
    c2_rrmse_avg = np.mean(np.ma.masked_invalid(c2_rrmse))
    c3_rrmse_avg = np.mean(np.ma.masked_invalid(c3_rrmse))

    return c1_rrmse, c1_rrmse_avg, c2_rrmse, c2_rrmse_avg, c3_rrmse, c3_rrmse_avg

def rmse_errors(color_space, array1, array2):
    if color_space == "Yiq":
        true = np.array([to_yiq(color) for color in array2])
        pred = np.array([to_yiq(color) for color in array1])  
    elif color_space == "Hsv":
        true = np.array([to_hsv(color) for color in array2])
        pred = np.array([to_hsv(color) for color in array1])  
    elif color_space == "Lab":
        true = np.array([to_lab(color) for color in array2])
        pred = np.array([to_lab(color) for color in array1]) 
    else:
        pass          
    c1_rmse = [rmse(true[i][0], pred[i][0]) for i in range(len(true))]
    c2_rmse = [rmse(true[i][1], pred[i][1]) for i in range(len(true))]
    c3_rmse = [rmse(true[i][2], pred[i][2]) for i in range(len(true))]
    c1_rmse_avg = np.mean(np.ma.masked_invalid(c1_rmse))
    c2_rmse_avg = np.mean(np.ma.masked_invalid(c2_rmse))
    c3_rmse_avg = np.mean(np.ma.masked_invalid(c3_rmse))

    return c1_rmse, c1_rmse_avg, c2_rmse, c2_rmse_avg, c3_rmse, c3_rmse_avg

def hsv():
    assignColor = []
    hue = np.linspace(0, 360-360/16, 16)
    saturation = np.linspace(60, 100, 4)
    value = np.linspace(40, 100, 4)

    for h in hue:
        for v in value:
            for s in saturation:
                RGB = hsv_to_rgb(h/360, s/100, v/100)
                RGB_C = [int(_*255) for _ in RGB]
                assignColor.append(RGB_C)

    return assignColor

def lab():
    L = np.linspace(30, 50, 4)
    A = np.linspace(-128, 128, 8)
    B = np.linspace(-128, 128, 8)
    colors = []
    for b in B:
        for a in A:
            for l in L:
                lab_color = LabColor(l, a, b)
                rgb_color = convert_color(lab_color, AdobeRGBColor)
                rgb_values = [int(rgb_color.clamped_rgb_r *255),
                                int(rgb_color.clamped_rgb_g*255), 
                                int(rgb_color.clamped_rgb_b*255)]
                colors.append(rgb_values)
    return colors

def yiq():
    y_levels = np.linspace(0.2, 0.6, 4)
    i_levels = np.linspace(-0.5, 0.5, 8)
    q_levels = np.linspace(-0.5, 0.5, 8)
    colors = []
    for i in i_levels:
        for q in q_levels:
            for y in y_levels:
                colors.append([int(i*255) for i in yiq_to_rgb(y, i, q)])
    return colors

def create_color_palette(color_space):
    if color_space == "Hsv":
        colors = hsv()
    elif color_space == "Lab":
        colors = lab()
    elif color_space == "Yiq":
        colors = yiq()
    else:
        pass
    w = h = int(np.sqrt(len(colors)))
    full_palette = np.zeros((h+2, w+2, 3), dtype = np.uint8)
    color_matrix = np.array(colors).reshape(h, w, 3)
    return color_matrix, colors

def image_segmentation(image_path):
    pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)
    mask = pipe(image_path, return_mask = True) 
    image_pillow = pipe(image_path)
    temp_image_path = "temp.png"
    image_pillow.save(temp_image_path)
    image = cv.imread(temp_image_path)
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    return image_rgb, mask

def CANNY_ED(img, sigma = 0.3):
    blur = cv.GaussianBlur(img, (5,5), cv.BORDER_CONSTANT)
    median = np.median(img)
    lower = int(max(0, (1.0-sigma)*median))
    upper = int(min(255, (1.0+sigma)*median))
    autocanny = cv.Canny(blur, lower, upper)
    return autocanny

def histogram_equalization(img_in):
    hsv_img = cv.cvtColor(img_in, cv.COLOR_RGB2HSV)
    h, s, v = cv.split(hsv_img)
    v_eq = cv.equalizeHist(v)
    hsv_eq_img = cv.merge((h, s, v_eq))
    img_out = cv.cvtColor(hsv_eq_img, cv.COLOR_HSV2RGB)
    return img_out

def percentile_whitebalance(image, percentile_value = 90):
        fig, ax = plt.subplots(1,2, figsize=(12,6))
        for channel, color in enumerate('rgb'):
            channel_values = image[:,:,channel]
            value = np.percentile(channel_values, percentile_value)
            ax[0].step(np.arange(256), 
                    np.bincount(channel_values.flatten(), 
                    minlength=256)*1.0 / channel_values.size, 
                    c=color)
            ax[0].set_xlim(0, 255)
            ax[0].axvline(value, ls='--', c=color)
            ax[0].text(value-70, .01+.012*channel, 
                    "{}_max_value = {}".format(color, value), 
                        weight='bold', fontsize=10)
            ax[0].set_xlabel('channel value')
            ax[0].set_ylabel('fraction of pixels')
            ax[0].set_title('Histogram of colors in RGB channels')    
            whitebalanced = img_as_ubyte(
                    (image*1.0 / np.percentile(image, 
                    percentile_value, axis=(0, 1))).clip(0, 1))
            ax[1].imshow(whitebalanced);
            ax[1].axis('off')
            ax[1].set_title('Whitebalanced Image')
        return ax, whitebalanced

def find_dominant_color(image_np, k=1):
    if image_np.ndim != 3 or image_np.shape[2] != 3:
        raise ValueError("Input must be an RGB image in numpy array format.")
    
    image_np = cv.cvtColor(image_np, cv.COLOR_RGB2BGR)
    pixels = image_np.reshape((-1, 3))
    pixels = np.float32(pixels)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centroids = cv.kmeans(pixels, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)
    dominant_color = centroids[0].astype(int)
    dominant_color = tuple(int(c) for c in dominant_color[::-1])
    
    return dominant_color

def find_average_color(image):
    mean = np.mean(image, axis=(0,1))
    std_dev = np.std(image, axis=(0,1))
    return mean, std_dev

def Color_Matrix_with_Border(array, w, h, ppm, bw=0, bc=(0,0,0)):
    obw = 2 * bw
    mw = ppm + bw
    tw = w * mw + 2 * obw  - bw
    th = h * mw + 2 * obw  - bw
    
    data = np.zeros((th, tw, 3), dtype=np.uint8)
    data[:] = bc

    for i, array in enumerate(array):
        row = (i // w) * mw + obw
        col = (i % w) * mw + obw
        data[row:row+ppm, col:col+ppm] = array
    
    return data

from colormath.color_objects import LabColor
from colormath.color_diff import delta_e_cie2000
import numpy

def patch_asscalar(a):
    return a.item()

setattr(numpy, "asscalar", patch_asscalar)

def lab_conv(rgb_triplet):
    r, g, b = [x / 255 for x in rgb_triplet] 
    rgb_color = AdobeRGBColor(r, g, b)
    lab_color = convert_color(rgb_color, LabColor)
    return lab_color

def delta_e_differences(true_lab, pred_lab):
    assert len(true_lab) == len(pred_lab), "True and predicted arrays must be of equal length."
    delta_e = [delta_e_cie2000(true_lab[i], pred_lab[i]) for i in range(len(true_lab))]
    return np.array(delta_e)

def percentage_below_threshold(items, threshold):
    count_below_threshold = sum(1 for item in items if item < threshold)
    total_items = len(items)
    
    if total_items == 0:
        return 0 
    percentage = (count_below_threshold / total_items) * 100
    return percentage

def stat1(color_space, array):
    if color_space == "Yiq":
        color_array = np.array([to_yiq(color) for color in array])
    elif color_space == "Hsv":
        color_array = np.array([to_hsv(color) for color in array])
    elif color_space == "Lab":
        color_array = np.array([to_lab(color) for color in array])
    else:
        pass  
    c1 = [i[0] for i in color_array]
    c2 = [i[1] for i in color_array]
    c3 = [i[2] for i in color_array]

    c1_m = np.mean(c1)
    c2_m = np.mean(c2)
    c3_m = np.mean(c3)
    c1_std = np.std(c1)
    c2_std = np.std(c2)
    c3_std = np.std(c3)

    return c1_m, c1_std, c2_m, c2_std, c3_m, c3_std

def stat2(array):
    lab_array = np.array([lab_conv(color) for color in array])
    n = int(np.sqrt(len(array)))
    lab_array = lab_array.reshape(n, n)

    delta_e_rows = []
    delta_e_cols = []

    for i in range(n):
        row_diffs = [delta_e_cie2000(lab_array[i, j], lab_array[i, j + 1]) for j in range(n - 1)]
        delta_e_rows.append(row_diffs)

        col_diffs = [delta_e_cie2000(lab_array[j, i], lab_array[j + 1, i]) for j in range(n - 1)]
        delta_e_cols.append(col_diffs)
    return delta_e_rows, np.mean(delta_e_rows), delta_e_cols, np.mean(delta_e_cols)


def results(color_space, ppm, specimen):
    image_path = f"/Users/rashid/Library/CloudStorage/OneDrive-UniversityofBristol/Year 3/University Work/Term 2/IRP/4. Decoding Information/Colored_QrCode_Decoding/Version 3/{color_space}/{color_space}_{ppm}_{specimen}.jpeg"
    img = cv.imread(image_path, 1)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(gray_img)
    blur = cv.GaussianBlur(cl1,(5,5),0)
    _,binarized_image = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    image, mask = image_segmentation(image_path)

    gray_img2 = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    clahe2 = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl2 = clahe2.apply(gray_img2)
    _,binarized_image2 = cv.threshold(cl2,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

    canny = CANNY_ED(binarized_image2)
    kernel1 = np.ones((5, 5), np.uint8) 
    img_dilation1 = cv.dilate(canny, kernel1, iterations=5)

    contours_dilate, _ = cv.findContours(img_dilation1,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE )
    contours_sorted = sorted(contours_dilate, key=cv.contourArea, reverse=True)
    c = contours_sorted[0]
    rect = cv.minAreaRect(c)
    box = cv.boxPoints(rect)
    box = np.array(box, dtype="int")
    ordered_box = np.zeros((4, 2), dtype="float32")
    s = box.sum(axis=1)
    ordered_box[0] = box[np.argmin(s)]
    ordered_box[2] = box[np.argmax(s)]
    diff = np.diff(box, axis=1)
    ordered_box[1] = box[np.argmin(diff)]
    ordered_box[3] = box[np.argmax(diff)]
    side = max([np.linalg.norm(ordered_box[i] - ordered_box[(i + 1) % 4]) for i in range(4)])
    dst_pts = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype="float32")
    M = cv.getPerspectiveTransform(ordered_box, dst_pts)
    warped = cv.warpPerspective(img, M, (int(side), int(side)))

    gray_img3 = cv.cvtColor(warped, cv.COLOR_BGR2GRAY)
    clahe3 = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl3 = clahe3.apply(gray_img3)
    _,binarized_image3 = cv.threshold(cl3,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

    height1, width1 = binarized_image3.shape
    coordinates = []
    for y in range(height1):
        for x in range(width1):
            if binarized_image3[y, x] == 0:
                coordinates.append((x, y))

    ordered_box1 = np.zeros((4, 2), dtype="float32")
    ordered_box1[2] = max(coordinates, key=lambda coord: coord[0] + coord[1])
    ordered_box1[0] = min(coordinates, key=lambda coord: coord[0] + coord[1])
    ordered_box1[1] = max(coordinates, key=lambda coord: coord[0] - coord[1])
    ordered_box1[3] = min(coordinates, key=lambda coord: coord[0] - coord[1])

    side1 = 29*50
    dst_pts1 = np.array([[0, 0], [side1 - 1, 0], [side1 - 1, side1 - 1], [0, side1 - 1]], dtype="float32")
    M1 = cv.getPerspectiveTransform(ordered_box1, dst_pts1)
    warped2 = cv.warpPerspective(warped, M1, (int(side1), int(side1)))

    hist_eq = histogram_equalization(warped2)
    ax, wb = percentile_whitebalance(hist_eq, 90)

    n = wb
    height, width, _ = n.shape
    num_parts = 29

    box_height = box_width = 50

    average_colors = []
    std_devs = []
    for i in range(num_parts):
        start_y = i * box_height
        end_y = min((i + 1) * box_height, height)

        for j in range(num_parts):
            start_x = j * box_width
            end_x = min((j + 1) * box_width, width)
            box = n[start_y:end_y, start_x:end_x]
            dominant_color, std_dev = find_average_color(box)
            average_colors.append(dominant_color)
            std_devs.append(std_dev)

    average_colors = np.array(average_colors)
    std_devs = np.array(std_dev)

    w, h = int(np.sqrt(len(average_colors))), int(np.sqrt(len(average_colors)))
    data = Color_Matrix_with_Border(average_colors, w, h, 1)
    img1 = np.array(Image.fromarray(data, 'RGB'))

    width2 = height2 = int(np.sqrt(len(average_colors)))
    col_ref = img1[width2 - 17:width2 -1, height2 - 17: height2-1]
    colors_det = col_ref.reshape((256, 3))
    palette, colors = create_color_palette(color_space)

    colors1 = average_colors
    array1 = colors_det 
    array2 = colors

    c1_rrmse, c1_rrmse_avg, c2_rrmse, c2_rrmse_avg, c3_rrmse, c3_rrmse_avg = rrmse_errors(color_space, array1, array2)
    csc = [char.upper() for char in color_space]
    decimal_list = np.arange(0, 256, 1)
    c1_rmse, c1_rmse_avg, c2_rmse, c2_rmse_avg, c3_rmse, c3_rmse_avg = rmse_errors(color_space, array1, array2)

    true_lab = np.array([lab_conv(color) for color in array2])
    pred_lab = np.array([lab_conv(color) for color in array1]) 

    delta_e = delta_e_differences(true_lab, pred_lab)
    data_e = delta_e.reshape(16, 16)
    data_c1 = np.array(c1_rrmse).reshape(16, 16)
    data_c2 = np.array(c2_rrmse).reshape(16, 16)
    data_c3 = np.array(c3_rrmse).reshape(16, 16)

    result_delta_e = percentage_below_threshold(delta_e, 10)

    c1_m1, c1_std1, c2_m1, c2_std1, c3_m1, c3_std1 = stat1(color_space, array1)
    c1_m2, c1_std2, c2_m2, c2_std2, c3_m2, c3_std2 = stat1(color_space, array2)

    r1, rm1, c1, cm1= stat2(array1)
    r2, rm2, c2, cm2 = stat2(array2)
    quantized_image = quantize_using_delta_e(img1, np.array(array1))


    version = 3
    file_path = "/Users/rashid/Library/CloudStorage/OneDrive-UniversityofBristol/Year 3/University Work/Term 2/IRP/3. Encoding Information/Colored_QrCode_Encoding/sample4.aac"
    cqr = CQrCode(version, "hsv", file_path)
    matrix = cqr._isfunction

    modules = 0
    positions = []
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] == 255:
                modules += 1
                positions.append((i,j))
            else:
                pass

    data_colors = []
    for i in positions:
        data_colors.append(quantized_image[i[0]][i[1]])
    data_colors = [list(i) for i in data_colors]
    array1 = [[i[0], i[1], i[2]] for i in list(array1)]
    data_decimal = []


    for i in data_colors:
        decimal = array1.index(i)
        data_decimal.append(decimal)
    original_data = cqr._read_data(file_path)

    original_data = original_data[:len(data_decimal)]

    matched = 0

    for i in range(len(original_data)):
        if original_data[i] == data_decimal[i]:
            matched += 1
        else:
            pass

    success = (matched/len(original_data))*100
    rrmse_modules = []
    for i in range(len(original_data)):
        r = calculate_rrmse(data_decimal[i], original_data[i])
        rrmse_modules.append(r)
        
    rrmse_modules = [i for i in rrmse_modules if i is not None]
    results_rrmse =  percentage_below_threshold(rrmse_modules, 1)
    k1_c1 = c1_rrmse_avg
    k1_c2 = c2_rrmse_avg
    k1_c3 = c3_rrmse_avg
    k1 = [k1_c1, k1_c2, k1_c3]

    k2 = result_delta_e

    k3_c1_m = c1_m1
    k3_c1_std = c1_std1
    k3_c1_m_true = c1_m2
    k3_c1_std_true = c1_std2
    k3_c2_m = c2_m1
    k3_c2_std = c2_std1
    k3_c2_m_true = c2_m2
    k3_c2_std_true = c2_std2
    k3_c3_m = c3_m1
    k3_c3_std = c3_std1
    k3_c3_m_true = c3_m2
    k3_c3_std_true = c3_std2

    k3 = [[k3_c1_m, k3_c1_std, k3_c1_m_true, k3_c1_std_true], 
          [k3_c2_m, k3_c2_std, k3_c2_m_true, k3_c2_std_true],
          [k3_c3_m, k3_c3_std, k3_c3_m_true, k3_c3_std_true]]

    k4_rm = rm1
    k4_cm = cm1
    k4_rm_true = rm2
    k4_c_true = cm2

    k4 = [k4_rm, k4_cm, k4_rm_true, k4_c_true]

    k5 = success
    
    k6 = results_rrmse
    
    return k1, k2, k3, k4, k5, k6
            

In [ ]:
def append_results(data_structure, color_space, ppm_values, specimens_per_ppm):
    for ppm in ppm_values:
        for specimen in range(1, specimens_per_ppm + 1):
            k1, k2, k3, k4, k5, k6 = results(color_space, ppm, specimen)
            data_structure[color_space][ppm][specimen]['Key Metric 1'] = {
                'C1': {'Value': k1[0]},
                'C2': {'Value': k1[1]},
                'C3': {'Value': k1[2]}
            }
            data_structure[color_space][ppm][specimen]['Key Metric 2'] = k2
            data_structure[color_space][ppm][specimen]['Key Metric 3'] = {
                'C1': {'Mean': k3[0][0], 'Std': k3[0][1], 'True Mean': k3[0][2], 'True Std': k3[0][3]},
                'C2': {'Mean': k3[1][0], 'Std': k3[1][1], 'True Mean': k3[1][2], 'True Std': k3[1][3]},
                'C3': {'Mean': k3[2][0], 'Std': k3[2][1], 'True Mean': k3[2][2], 'True Std': k3[2][3]}
            }
            data_structure[color_space][ppm][specimen]['Key Metric 4'] = {
                'Row': {'Mean': k4[0], 'True Mean': k4[2]},
                'Column': {'Mean': k4[1], 'True Mean': k4[3]}
            }
            data_structure[color_space][ppm][specimen]['Key Metric 5'] = k5
            data_structure[color_space][ppm][specimen]['Key Metric 6'] = k6



data_structure = create_data_structure(color_spaces, ppm_values, specimens_per_ppm, metrics_info)

for color_space in color_spaces:
    append_results(data_structure, color_space, ppm_values, specimens_per_ppm)


In [ ]:
import pandas as pd
import numpy as np

def flatten_for_excel(data_structure):
    def unpack_metrics(row, key_prefix, value):
        """ Recursively unpack metrics into the row dictionary with flattened keys. """
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                new_key_prefix = f"{key_prefix} {sub_key}".strip()
                unpack_metrics(row, new_key_prefix, sub_value)
        else:
            row[key_prefix] = value
    
    rows = []
    for color_space, ppm_data in data_structure.items():
        for ppm, metrics_data in ppm_data.items():
            for specimen, metrics in metrics_data.items():
                row = {
                    'Color Space': color_space,
                    'PPM': ppm,
                    'Specimen': specimen
                }
                for metric_key, metric_values in metrics.items():
                    unpack_metrics(row, metric_key, metric_values)
                rows.append(row)
    return rows


flattened_data = flatten_for_excel(data_structure)
df = pd.DataFrame(flattened_data)

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(value="N/A", inplace=True)

def export_to_excel(df, file_name='data_structure.xlsx'):
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Detailed Metrics')

export_to_excel(df)